# Patient ID 7669

In [20]:
# original path from server "/projects/ps-gleesonlab8/User/chchung/Interneuron/20221223_7669_Ampliseq_QC/20230101_QC/final_summary_anno3.txt"
library(ggplot2)
library(dplyr)
library(tidyr)
library(ggcorrplot)
library(gplots)
library(reshape2)
library(corrplot)
library(grid)
library(pheatmap)
library(ggdendro)
library(heatmaply)
#library(devtools) 
library(ComplexHeatmap) #ComplexHeatmap cite "Gu, Z. (2022) Complex Heatmap Visualization, iMeta. DOI: 10.1002/imt2.43."
library(circlize)
library(dendsort)

In [6]:
filterdata <- read.table("../20230101_QC/final_summary_anno3.txt",header=T,sep="\t")
annotated <- read.table("../variant_annotation/780_variant_annotation.csv",header=T,sep=",",row.names=1)

# filterdata <- read.table("/Users/rahulnedunuri/Documents/GleesonLab/Mosaic Variant Projects/final_summary_anno3.txt" ,header=T,sep="\t")
# annotated <- read.csv("/Users/rahulnedunuri/Documents/GleesonLab/Mosaic Variant Projects/780_variant_annotation.csv" ,header=T,sep=",")

In [7]:
names(filterdata)
names(annotated)

[1] "ID"                        "CHROM"                    
 [3] "POS"                       "REF"                      
 [5] "ALT"                       "CHROM.POS.REF.ALT"        
 [7] "ANNO"                      "GENE"                     
 [9] "GNOMAD_FREQ"               "REPEAT_MASKER"            
[11] "SEGDUP"                    "HOMOPOLYMER"              
[13] "REF_SEQ"                   "DINUCLEOTIDE"             
[15] "NEAR_INDEL"                "UCSC_RPMSK"               
[17] "REF_COUNT"                 "ALT_COUNT"                
[19] "MAF"                       "LOWER_CI"                 
[21] "UPPER_CI"                  "CI_IS_GREATER"            
[23] "NORMAL_REF_COUNT"          "NORMAL_ALT_COUNT"         
[25] "NORMAL_MAF"                "NORMAL_LOWER_CI"          
[27] "NORMAL_UPPER_CI"           "NORMAL_CI_IS_GREATER"     
[29] "Mosaic"                    "Mosaic_shared_btw_WGS_Amp"
[31] "Hemisphere"                "Region"                   
[33] "Punch"                     "Cell_Type"                
[35] "Location"                  "AntPost"                  
[37] "DorVen"                    "Organ"                    
[39] "Note"

[1] "variant_list"            "sample_number_presented"
 [3] "HEART"                   "ADRENAL"                
 [5] "CB"                      "CTX"                    
 [7] "LIVER"                   "KIDNEY"                 
 [9] "POA"                     "BG"                     
[11] "THAL"                    "HIP"                    
[13] "SKIN"                    "OLF"                    
[15] "L"                       "R"                      
[17] "DLX1"                    "TBR1"                   
[19] "COUPTF2"                 "Bulk"                   
[21] "NeuN"                    "Brain_only"             
[23] "CTX_only"                "CTX_BG_only"            
[25] "BG_only"                 "THAL_only"              
[27] "Left_only"               "Right_only"             
[29] "IN_only"                 "COUP_only"              
[31] "TBR_only"                "TBR_BR"                 
[33] "DLX_BR"

## Filter for Mosaic Variants Shared by WGS and Amplicon Sequencing

In [8]:
#sort df for only WGS + Ampliseq confirmed (1) -> confirmed variants (excluding JGG controls)
totalConfVarsInclCtrl = length(unique( subset(filterdata, Mosaic_shared_btw_WGS_Amp == 1) ))
confirmedVars = subset(filterdata, Mosaic_shared_btw_WGS_Amp == 1 & Organ != "JGG")

numTotalVars = length(unique((confirmedVars$CHROM.POS.REF.ALT)))
pretext = "Confirmed total variants (WGS + Amp): "
print(paste(pretext, length(unique((confirmedVars$CHROM.POS.REF.ALT))))) 

[1] "Confirmed total variants (WGS + Amp):  780"


## Sorting of Confirmed Mosaic Variants by Region and MAF_CI within Mosaic Threshold

In [9]:
#Mosaic Variants only in the brain
#Brain : CTX, BG, THAL, HIP, POA (preoptic area), OLF, CB (cerebellum)
#Kidney : ADRENAL, KIDNEY
#Other: HEART, LIVER, SKIN
# Exculde JGG variants (control)


allOrgans = unique(confirmedVars$Organ)
#print("All regions: ")
#print(allOrgans)

##problem, all are 743; use MAF above threshold
##upper and lower bounds for MAF to be a valid variant
Mosaic_UPPER = 0.4562841
Mosaic_LOWER = 0.002360687

brainVars = subset(confirmedVars, (Organ == "CTX" | Organ == "BG" | Organ == "THAL" | Organ == "HIP" | Organ == "POA" | Organ == "CB" | Organ == "OLF"))
brainVars = subset(brainVars, LOWER_CI > Mosaic_LOWER & UPPER_CI < Mosaic_UPPER & NORMAL_LOWER_CI < Mosaic_LOWER)
numBrainUniqueVars = length(unique(brainVars$CHROM.POS.REF.ALT))

leftHemisphereVars = subset(brainVars, Hemisphere == "L")
leftHemisphereVars = subset(leftHemisphereVars, LOWER_CI > Mosaic_LOWER & UPPER_CI < Mosaic_UPPER & NORMAL_LOWER_CI < Mosaic_LOWER)
numLeftHVars = length(unique(leftHemisphereVars$CHROM.POS.REF.ALT))

rightHemisphereVars = subset(brainVars, Hemisphere == "R")
rightHemisphereVars = subset(rightHemisphereVars, LOWER_CI > Mosaic_LOWER & UPPER_CI < Mosaic_UPPER & NORMAL_LOWER_CI < Mosaic_LOWER)
numRightHVars = length(unique(rightHemisphereVars$CHROM.POS.REF.ALT))

cortexVars = subset(brainVars, Organ == "CTX")
cortexVars = subset(cortexVars, LOWER_CI > Mosaic_LOWER & UPPER_CI < Mosaic_UPPER & NORMAL_LOWER_CI < Mosaic_LOWER)
numCortexVars = length(unique(cortexVars$CHROM.POS.REF.ALT))

basalGangliaVars = subset(brainVars, Organ == "BG")
basalGangliaVars = subset(basalGangliaVars, LOWER_CI > Mosaic_LOWER & UPPER_CI < Mosaic_UPPER & NORMAL_LOWER_CI < Mosaic_LOWER)
numBgVars = length(unique(basalGangliaVars$CHROM.POS.REF.ALT))

dlx1Vars = subset(brainVars, Cell_Type == "DLX1")
dlx1Vars = subset(dlx1Vars, LOWER_CI > Mosaic_LOWER & UPPER_CI < Mosaic_UPPER & NORMAL_LOWER_CI < Mosaic_LOWER)
numDlx1Vars = length(unique(dlx1Vars$CHROM.POS.REF.ALT))

tbr1Vars = subset(brainVars, Cell_Type == "TBR1")
tbr1Vars = subset(tbr1Vars, LOWER_CI > Mosaic_LOWER & UPPER_CI < Mosaic_UPPER & NORMAL_LOWER_CI < Mosaic_LOWER)
numTbr1Vars = length(unique(tbr1Vars$CHROM.POS.REF.ALT))

dlx1_couptf2Vars = subset(brainVars, Cell_Type == "DLX1" | Cell_Type == "COUPTF2")
dlx1_couptf2Vars = subset(dlx1_couptf2Vars, LOWER_CI > Mosaic_LOWER & UPPER_CI < Mosaic_UPPER & NORMAL_LOWER_CI < Mosaic_LOWER)
numdlx1_couptf2Vars = length(unique(dlx1_couptf2Vars$CHROM.POS.REF.ALT))



print(paste(numTotalVars, "total confirmed unique variants"))
print(paste(numBrainUniqueVars, "unique variants in brain"))
print(paste(numLeftHVars, "unique variants in left hemisphere"))
print(paste(numRightHVars, "unique variants in right hemisphere"))
print(paste(numCortexVars, "unique variants in cortex"))
print(paste(numBgVars, "unique variants in basal ganglia"))
print(paste(numDlx1Vars, "unique variants in DLX1 cells"))
print(paste(numTbr1Vars, "unique variants in TBR1 cells"))
print(paste(numdlx1_couptf2Vars, "unique variants in DLX1+COUPTFII cells (GABAergic interneurons)"))


[1] "780 total confirmed unique variants"
[1] "428 unique variants in brain"
[1] "264 unique variants in left hemisphere"
[1] "282 unique variants in right hemisphere"
[1] "201 unique variants in cortex"
[1] "178 unique variants in basal ganglia"
[1] "190 unique variants in DLX1 cells"
[1] "156 unique variants in TBR1 cells"
[1] "194 unique variants in DLX1+COUPTFII cells (GABAergic interneurons)"


## Bargraph of Number of Unique Mosaic Variants By Region

In [10]:

variantCounts = c(numBrainUniqueVars, numCortexVars, numTotalVars, numLeftHVars, numRightHVars, numBgVars)
regions = c("Brain", "Cortex", "Total", "Left H", "Right H", "Basal Ganglia")
variantCountsByCellType = c(numDlx1Vars, numTbr1Vars, numdlx1_couptf2Vars)
cellTypes = c("DLX1", "TBR1", "DLX1+COUPTF2")

variantsByRegion = data.frame(regions, variantCounts)
variantsByCellType = data.frame(cellTypes, variantCountsByCellType)

regionCount <- ggplot(variantsByRegion, aes(x = regions, y = variantCounts)) +
  geom_bar(stat = "identity") +
  theme(panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(),
        panel.background = element_rect(fill = "white"),
        axis.line = element_line()) + 
  geom_text(aes(label = variantCounts), position = position_dodge(0.9), vjust = -0.5, color = "#940303")

cellTypeCount <- ggplot(variantsByCellType, aes(x = cellTypes, y = variantCountsByCellType)) +
  geom_bar(stat = "identity") +
  theme(panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank(),
        panel.background = element_rect(fill = "white"),
        axis.line = element_line()) + 
  geom_text(aes(label = variantCountsByCellType), position = position_dodge(0.9), vjust = -0.5, color = "#940303")

#print(regionCount + labs(title = "Unique Variant Counts by Region", x = "Region", y = "Count"))
#print(cellTypeCount + labs(title = "Unique Variant Counts by Cell Type", x = "Cell Type", y = "Count"))

## Correlation matrix of Variant x Variant using pheatmap

In [11]:
#function for pheatmap saving
save_pheatmap_pdf <- function(x, filename, width=10, height=10) {
   stopifnot(!missing(x))
   stopifnot(!missing(filename))
   pdf(filename, width=width, height=height)
   grid::grid.newpage()
   grid::grid.draw(x$gtable)
   dev.off()
}

In [16]:
#ComplexHeatmap plot (working)
#Left/Right PEARSON

varsConfirmedIn2Sample = subset(annotated, sample_number_presented >= 2)

confirmed2SampleVars = subset(confirmedVars, CHROM.POS.REF.ALT %in% varsConfirmedIn2Sample$variant_list)

onlyMAFofVars = split(sqrt(confirmed2SampleVars$MAF), confirmed2SampleVars$CHROM.POS.REF.ALT) #now using sqrt(MAF) 4/14/23
df_encoded <- model.matrix(~ . - 1, data = onlyMAFofVars) #converts to matrix format with col variants, row MAF
pearson_corr_matrix = round(cor(df_encoded, method = 'pearson', use = 'pairwise.complete.obs'), 4)

LR_colors = colorRamp2(c(-1, 0, 1), c("#d68426", "#f5f0f0", "#2a1abf"))

pdf("220_correlation_LEFT_RIGHT_Pearson.pdf")


varsConfirmedIn2Sample = varsConfirmedIn2Sample[order(varsConfirmedIn2Sample$X), ]

#print(subset(annotated, variant_list == "X-36528250-G-T"))
left_right_cell_col_ann = rowAnnotation(LR_only = varsConfirmedIn2Sample$Left_only - varsConfirmedIn2Sample$Right_only, 
                                        COUP_only = varsConfirmedIn2Sample$COUP_only,
                                        IN_only = varsConfirmedIn2Sample$IN_only,
                                        TBR_only = varsConfirmedIn2Sample$TBR_only,
                                        TBR_BR = varsConfirmedIn2Sample$TBR_BR,
                                        DLX_BR = varsConfirmedIn2Sample$DLX_BR,
                                        annotation_name_gp = gpar(fontsize = 8),
                                        col = list(LR_only = LR_colors),
                                        annotation_legend_param = list(
                                                LR_only = list(
                                                        title = "Left/Right Only",
                                                        at = c(-1, 0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("Right", "Both", "Left")),
                                                COUP_only = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "COUPTF2 only")),
                                                IN_only = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "Interneuron only")),
                                                TBR_only = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "TBR1 only")),
                                                TBR_BR = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "TBR1 Brain only")),
                                                DLX_BR = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "DLX1 Brain only")))) 

leftMapUpdate = Heatmap(pearson_corr_matrix,
        col = colorRamp2(c(-1, 0, 1), c("#0d0dad", "white", "firebrick3")),
        row_names_gp = gpar(fontsize = "2"), column_names_gp = gpar(fontsize = "2"), name = "Correlation",
        left_annotation = left_right_cell_col_ann,
        clustering_method_rows = "complete", clustering_method_columns = "complete", 
        clustering_distance_rows = "euclidean", clustering_distance_columns = "euclidean")
leftMapUpdate
dev.off()

ERROR: Error in order(varsConfirmedIn2Sample$X): argument 1 is not a vector


In [133]:
#Left/Right SPEARMAN only

pdf("220_correlation_LEFT_RIGHT_Spearman.pdf")


spearman_corr_matrix = round(cor(df_encoded, method = 'spearman', use = 'pairwise.complete.obs'), 4)

varsConfirmedIn2Sample = varsConfirmedIn2Sample[order(varsConfirmedIn2Sample$X), ]

left_right_cell_col_ann = rowAnnotation(LR_only = varsConfirmedIn2Sample$Left_only - varsConfirmedIn2Sample$Right_only, 
                                        COUP_only = varsConfirmedIn2Sample$COUP_only,
                                        IN_only = varsConfirmedIn2Sample$IN_only,
                                        TBR_only = varsConfirmedIn2Sample$TBR_only,
                                        TBR_BR = varsConfirmedIn2Sample$TBR_BR,
                                        DLX_BR = varsConfirmedIn2Sample$DLX_BR,
                                        annotation_name_gp = gpar(fontsize = 8),
                                        col = list(LR_only = LR_colors),
                                        annotation_legend_param = list(
                                                LR_only = list(
                                                        title = "Left/Right Only",
                                                        at = c(-1, 0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("Right", "Both", "Left")),
                                                COUP_only = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "COUPTF2 only")),
                                                IN_only = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "Interneuron only")),
                                                TBR_only = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "TBR1 only")),
                                                TBR_BR = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "TBR1 Brain only")),
                                                DLX_BR = list(
                                                        title = "",
                                                        at = c(0, 1),
                                                        color_bar = "discrete",
                                                        labels = c("", "DLX1 Brain only")))) 


row_means = rowMeans(spearman_corr_matrix)
col_means = colMeans(spearman_corr_matrix)

row_dend = as.dendrogram(hclust(dist(spearman_corr_matrix)))
col_dend = as.dendrogram(hclust(dist(t(spearman_corr_matrix))))

row_dend = dendsort(row_dend, type="average")
col_dend = dendsort(col_dend, type="average")

leftMapUpdate = Heatmap(spearman_corr_matrix,
        col = colorRamp2(c(-1, 0, 1), c("#0d0dad", "white", "firebrick3")),
        row_names_gp = gpar(fontsize = "1"), column_names_gp = gpar(fontsize = "1"), name = "Correlation",
        left_annotation = left_right_cell_col_ann,
        clustering_method_rows = "complete", clustering_method_columns = "complete", 
        clustering_distance_rows = "euclidean", clustering_distance_columns = "euclidean",
        cluster_rows = TRUE, cluster_columns = TRUE) 
leftMapUpdate
dev.off()


pdf 
  2